In [1]:
import pandas as pd
import glob2
import matplotlib.pyplot as plt
import datetime
import re
from datetime import datetime, timedelta
import ChartsAnalysisALL_Functions

In [2]:
path_to_test_config1 = "../configs/chartsanalysisparams.yml"
test_config1 = ChartsAnalysisALL_Functions.load_test_config(path_to_test_config1)
thelocalenames = test_config1["chartdetails"]["locales"]
read_enc_vals = test_config1["chartdetails"]["read_enc_val"]

In [3]:
thelocalenames

['kr', 'za', 'global', 'gb', 'ng', 'au', 'in', 'us']

In [4]:
print("STARTING THE RUN......................................................")

read_enc_val = read_enc_vals[0]

x = datetime.now()

thelogdetails = ""
thelogdetails = "The run datetime is: " + str(x)

STARTING THE RUN......................................................


In [19]:
for locale_name in thelocalenames:
    print("THE CURRENT CHART LOCALE RUNNING IS: ", str(locale_name))

    thelogdetails = thelogdetails + "THE CURRENT CHART LOCALE RUNNING IS: " + str(locale_name) + "\n"

    global_music_file_paths = glob2.glob("../"+'spotify ' + locale_name + ' v2/*.csv')

    thelogdetails = thelogdetails + " The number of files is: " + str(len(global_music_file_paths)) + "\n"
    print(len(global_music_file_paths))

    theglobal_music_file_path = [global_music_file_paths[len(global_music_file_paths)-1]]

    ChartsAnalysisALL_Functions.addsDatesToData(locale_name,theglobal_music_file_path)

    print(theglobal_music_file_path)

    #### Add Dates to the Files
    
    #Combine the files and sort by dates
    all_files = []
    # trouble_files = []
    for file in theglobal_music_file_path:
        the_data_in_file = pd.read_csv(file)
        
        if("URL" in list(the_data_in_file.columns)):
        
            the_data_in_file = the_data_in_file.drop(["URL"], axis=1)
        
        list_vals = sum(list(the_data_in_file.isnull().sum()))
        
        # if(list_vals > 0):
        #     trouble_files.append(file)
        all_files.append(the_data_in_file)
        
    the_spotify_data = pd.concat(all_files) 

    # trouble_files_df = pd.concat(trouble_files)
    
    all_cols = list(the_spotify_data.columns)
    drop_cols = []
    
    for col in all_cols:
        if("Unnamed" in col or "artificial" in col):
            drop_cols.append(col)
            
    the_spotify_data.drop(drop_cols, axis=1, inplace = True)
    
    ### Error checking that all the Thursday dates are present
    date_list = the_spotify_data["End Date"].unique().tolist()
    print("DATE LIST....", date_list)
    
    # ### Continue
    
    the_spotify_data["Week"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
    the_spotify_data["Year"] = the_spotify_data["Week"]
    
    the_spotify_data['Week'] = the_spotify_data['Week'].apply(lambda x : x.isocalendar()[1] )
    the_spotify_data['Year'] = the_spotify_data['Year'].apply(lambda x : x.isocalendar()[0] )
    
    # the_spotify_data["Start Date Dt"] = the_spotify_data["Start Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
    the_spotify_data["End Date Dt"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
    the_spotify_data['Seconds since Epoch'] = the_spotify_data['End Date Dt'].apply(lambda x : round(x.timestamp(), 0))


    the_spotify_data["End Date Aggregated"] = the_spotify_data["End Date"].apply(lambda x : ChartsAnalysisALL_Functions.changedaytotheFirst(x) )

    #find the number of artists on the track
    the_spotify_data.dropna(inplace=True)
    
    the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
    the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
    the_spotify_data["artist_names"] = the_spotify_data.apply(lambda x : x["artist_names"] + "Artist_NA" if(x["artist_names"] == "") else x["artist_names"], axis=1)
    
    the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
    the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
    the_spotify_data["track_name"] = the_spotify_data.apply(lambda x : x["artist_names"] + "_NA" if(x["track_name"] == "") else x["track_name"], axis=1)

    the_spotify_data['ArtistCount'] = the_spotify_data['track_name'].apply(lambda x : ChartsAnalysisALL_Functions.countNumberArtists(x))

    print(the_spotify_data.shape)
    
    the_spotify_data["main_artist"] = the_spotify_data.apply(lambda x : x["artist_names"].split(",")[0], axis=1)
    
    the_spotify_data = the_spotify_data.reset_index()
    the_spotify_data.drop(["index"], axis=1, inplace=True)
    
    print(the_spotify_data.shape)

    the_spotify_data = the_spotify_data[~the_spotify_data.index.duplicated()]
    the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : ChartsAnalysisALL_Functions.everInTopTen(x))
    
    # the_spotify_data.head()
    
    ### When did the track reach the Top 10 
    
    ### how long does a song last on the charts?
    
    # the_spotify_data["isTopTen"].value_counts()
    
    # the_spotify_data.isnull().sum()
    
    unique_artists = the_spotify_data["main_artist"].unique().tolist()
    num_artists = len(unique_artists)
    print("THE NUM OF ARTISTS: ", num_artists)
    thelogdetails = thelogdetails + " The number of artists in the data: " + str(num_artists) + "\n"

    the_spotify_data = the_spotify_data.drop(['artist_names', 'track_name'], axis=1)
    the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : ChartsAnalysisALL_Functions.getArtistAppearanceCount(x))
    the_spotify_data.drop(["level_2"], inplace=True, axis=1)
    

    the_spotify_data =the_spotify_data.groupby(["artist_names", "track_name"], as_index=False).apply(lambda x : ChartsAnalysisALL_Functions.getTrackAppearanceCount(x))
    # the_spotify_data = the_spotify_data.reset_index()
    the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)
    # print(the_spotify_data.columns)
    # stop
    
    # the_spotify_data.head()
    
    the_spotify_data.to_csv("the_spotify_data_" + locale_name+ ".csv", index=False)
    
    the_spotify_data["rank"] = the_spotify_data["rank"].astype(int)
    the_spotify_data["streams"] = the_spotify_data["streams"].astype(int)
    the_spotify_data["ArtistCount"] = the_spotify_data["ArtistCount"].astype(int)
    
    the_spotify_data["isTopTen"] = the_spotify_data["isTopTen"].astype(int)
    # the_spotify_data["lenOnCharts"] = the_spotify_data["lenOnCharts"].astype(int)
    the_spotify_data["Year"] = the_spotify_data["Year"].astype(int)
    the_spotify_data["Week"] = the_spotify_data["Week"].astype(int)
    
    all_cols = list(the_spotify_data.columns)
    drop_cols = []
    
    for col in all_cols:
        if("Unnamed" in col or "artificial" in col):
            drop_cols.append(col)
            
    the_spotify_data.drop(drop_cols, axis=1, inplace = True)
    
    the_spotify_data["Artist and Track"] = the_spotify_data["artist_names"] + "; " + the_spotify_data["track_name"]
    
    ### Getting the list of Artists and Tracks
    all_files_grouped = the_spotify_data.groupby(["main_artist", "track_name"]).apply(lambda x: ChartsAnalysisALL_Functions.trackAppearance(x))
    
    # all_files_grouped["rank difference"].max()

    all_files_grouped["Position over Time"] = all_files_grouped.apply(lambda x : ChartsAnalysisALL_Functions.positionvertime(x), axis=1)

    ##### Add a Girl Group and Boy Group
    
    ggdf = pd.read_csv("../Classification/ListofGirlGroups.csv", encoding='ANSI')
    bgdf = pd.read_csv("../Classification/ListofBoyGroups.csv", on_bad_lines='skip', encoding='ANSI')
    mgdf = pd.read_csv("../Classification/ListofMixedGroups.csv", on_bad_lines='skip', encoding='ANSI')

    all_files_grouped["IsGirlGroup"] = all_files_grouped["main_artist"].apply(lambda x : ChartsAnalysisALL_Functions.IsGG(ggdf, x))
    all_files_grouped["IsBoyGroup"] = all_files_grouped["main_artist"].apply(lambda x : ChartsAnalysisALL_Functions.IsGG(bgdf, x))
    all_files_grouped["IsMixedGroup"] = all_files_grouped["main_artist"].apply(lambda x : ChartsAnalysisALL_Functions.IsGG(mgdf, x))
    
    ggs = all_files_grouped[all_files_grouped["IsGirlGroup"]==True]["main_artist"].unique().tolist()
    bgs = all_files_grouped[all_files_grouped["IsBoyGroup"]==True]["main_artist"].unique().tolist()
    
    print(all_files_grouped.columns)
    print("the output path......", "../Classification/all_files_"+locale_name+"v1.csv")

    print("the grped file shape: ", all_files_grouped.shape)

    # ..................................................................................................................
    thefile = "../Classification/all_files_"+locale_name+"v1.csv"
    all_files = []

    all_the_spotify_data = pd.read_csv(thefile)

    combined_files = pd.concat([all_the_spotify_data, all_files_grouped])
    
    print("THE SHAPE: ", all_the_spotify_data.shape)
    print("THE COMPLETE SHAPE: ", combined_files.shape)
    
    # print(all_the_spotify_data)
    date_list = combined_files["End Date"].unique().tolist()

    thelogdetails = thelogdetails + " The number of dates is: " + str(len(date_list)) + "\n"
    start_date = date_list[0]
    end_date = date_list[len(date_list)-1]

    # print("END DATE" , date_list)
    
    # len(date_list)

    thelogdetails = thelogdetails + " The start dates is: " + str(start_date) + "." + " The end date is: " + str(end_date) + "\n"
    print("THE START AND END DATES ARE: ", start_date, end_date)

    result = ChartsAnalysisALL_Functions.check_thursdays_in_range(date_list, start_date, end_date)
    print("All Thursdays present:", result)
    thelogdetails = thelogdetails + " All Thursdays present: " + str(result) + "\n"

    #------------------------------------------------------
    thelogdetails = thelogdetails + " The output data written to this file: " + "Classification/all_files_"+locale_name+"v1.csv" + "\n"
    with open("../Classification/Spotify_Logs.txt", "a") as f:
        f.write(thelogdetails)

    # combined_files.to_csv("../Classification/all_files_"+locale_name+"v1.csv", index=False)
    
    print("DONE.")

THE CURRENT CHART LOCALE RUNNING IS:  kr
255
len to remove front 36
len to remove back 4
['../spotify kr v2\\regional-kr-weekly-2025-12-18.csv']
DATE LIST.... ['2025-12-18']
(200, 16)
(200, 17)


C:\Users\lkhum\AppData\Local\Temp\ipykernel_14824\350330892.py:90: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : ChartsAnalysisALL_Functions.everInTopTen(x))


THE NUM OF ARTISTS:  122


C:\Users\lkhum\AppData\Local\Temp\ipykernel_14824\350330892.py:108: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : ChartsAnalysisALL_Functions.getArtistAppearanceCount(x))
C:\Users\lkhum\AppData\Local\Temp\ipykernel_14824\350330892.py:112: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_da

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup', 'IsMixedGroup'],
      dtype='object')
the output path...... ../Classification/all_files_krv1.csv
the grped file shape:  (200, 26)
THE SHAPE:  (51000, 26)
THE COMPLETE SHAPE:  (51200, 26)
THE START AND END DATES ARE:  2025-06-12 2021-12-23
thursdays_in_range:  0
date_set:  255
All Thursdays are present in the range.
All Thursdays present: True
DONE.
THE CURRENT CHART LOCALE RUNNING IS:  za
406
len to remove front 36
len to remove back 4
['../spotify za v2\\regional-za-weekly-2025-12-18.csv']
DATE LIST.... ['2025-12-18']
(200, 16)
(200, 17)
THE NUM OF ARTISTS:  129


...

THE NUM OF ARTISTS:  89


C:\Users\lkhum\AppData\Local\Temp\ipykernel_14824\350330892.py:108: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : ChartsAnalysisALL_Functions.getArtistAppearanceCount(x))
C:\Users\lkhum\AppData\Local\Temp\ipykernel_14824\350330892.py:112: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_da

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup', 'IsMixedGroup'],
      dtype='object')
the output path...... ../Classification/all_files_inv1.csv
the grped file shape:  (200, 26)
THE SHAPE:  (68368, 26)
THE COMPLETE SHAPE:  (68568, 26)
THE START AND END DATES ARE:  2024-08-01 2020-03-26
thursdays_in_range:  0
date_set:  356
All Thursdays are present in the range.
All Thursdays present: True
DONE.
THE CURRENT CHART LOCALE RUNNING IS:  us
469
len to remove front 36
len to remove back 4
['../spotify us v2\\regional-us-weekly-2025-12-18.csv']
DATE LIST.... ['2025-12-18']
(200, 16)
(200, 17)
THE NUM OF ARTISTS:  128


C:\Users\lkhum\AppData\Local\Temp\ipykernel_14824\350330892.py:90: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : ChartsAnalysisALL_Functions.everInTopTen(x))
C:\Users\lkhum\AppData\Local\Temp\ipykernel_14824\350330892.py:108: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  the_spotify_data =the_spotify_data.groupby(["

Index(['trackAppearanceCount', 'artistAppearanceCount', 'artist_names',
       'track_name', 'rank', 'uri', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup', 'IsMixedGroup'],
      dtype='object')
the output path...... ../Classification/all_files_usv1.csv
the grped file shape:  (200, 26)
THE SHAPE:  (93795, 26)
THE COMPLETE SHAPE:  (93995, 26)
THE START AND END DATES ARE:  2024-02-15 2025-12-04
thursdays_in_range:  95
date_set:  469
All Thursdays are present in the range.
All Thursdays present: True
DONE.
